In [56]:
import glob 
from monai.transforms import (Compose, LoadImage, EnsureType, ScaleIntensityRange, Resize)
import torch
import numpy as np

In [59]:
loader = Compose([
    LoadImage(ensure_channel_first=True),
    EnsureType(device=None, track_meta=False),
]
)


data_path = glob.glob('/root/Competitions/Kaggle/CXR(Pneumonia)/data/chest_xray/train/*/*')

arr = loader(data_path[0])[0]


torch.mean(arr), torch.max(arr), torch.min(arr)



(tensor(142.3909), tensor(255.), tensor(0.))

In [43]:
from tqdm import tqdm
scaler = ScaleIntensityRange(a_min=0., a_max=255., b_min=0., b_max=1., clip=True)

cnt = 0 
x = 0
y = 0 
mean = 0 
var = 0

for path in tqdm(data_path):
    
    data = loader(path)[0]
    # print( data.size())
    c, w, h = data.size()
    
    x += w
    y += h
    data = scaler(data)
    data = data.flatten()
    img_mean = np.mean(data)
    img_var  = np.var(data)
    
    L = len(data)
    cnt += L
    mean += L * img_mean
    var += (img_var + img_mean**2) * L    

100%|██████████| 5216/5216 [00:57<00:00, 91.21it/s] 


In [45]:

overall_mean = mean / cnt
overall_var = var / cnt - overall_mean**2
overall_var = np.sqrt(overall_var)

print(overall_mean, overall_var)
print(overall_mean/255, overall_var/255)
'''
normalization not first applied:
    mean, std         = 124.52527598781411 62.28363538584806
    mean/255, std/255 = 0.48833441563848673 0.24424955053273747
    
normalization first applied:
    mean, std         = 0.48833443344643174 0.2442495558063171
    mean/255, std/255 = 0.00191503699390757 0.0009578413953188
'''

print()

0.48833443344643174 0.2442495558063171
0.0019150369939075753 0.0009578413953188907



In [61]:
# validity_check #
from monai.transforms import NormalizeIntensity
scaler = Compose([
    ScaleIntensityRange(a_min=0., a_max=255., b_min=0., b_max=1., clip=True),
    NormalizeIntensity(subtrahend=0.48833443344643174, divisor=0.2442495558063171)
])

cnt = 0 
x = 0
y = 0 
mean = 0 
var = 0

for path in tqdm(data_path):
    
    data = loader(path)[0]
    c, w, h = data.size()
    
    x += w
    y += h
    data = scaler(data)
    data = data.flatten()
    img_mean = np.mean(data)
    img_var  = np.var(data)
    
    L = len(data)
    cnt += L
    mean += L * img_mean
    var += (img_var + img_mean**2) * L
    

overall_mean = mean / cnt
overall_var = var / cnt - overall_mean**2
overall_var = np.sqrt(overall_var)

print(overall_mean, overall_var)
print(overall_mean/255, overall_var/255)

100%|██████████| 5216/5216 [01:50<00:00, 47.13it/s]

-1.0127896285845308e-07 1.0000000131623565
-3.971724033664827e-10 0.003921568679068065
